In [5]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wkqtarhy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wkqtarhy
  Resolved https://github.com/huggingface/transformers to commit 851a4f7088281072bdcf95edaaf08c6984e28cfd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8002731 sha256=7eb83b7d5cb3af69070230dab6c35b646ed87d28d80de5351a1dcd074221c4e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-9_c40ksd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install accelerate -qq

In [3]:
!pip install SentencePiece -qq

In [4]:
!pip install protobuf -qq

In [5]:
!pip install bitsandbytes -qq

In [6]:
!pip install scipy -qq

In [1]:
import torch
import transformers

if torch.cuda.is_available():
    torch.set_default_device("cuda")

In [5]:
model = transformers.AutoModelForCausalLM.from_pretrained("microsoft/Orca-2-7b", device_map='auto', load_in_8bit=True)


config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "microsoft/Orca-2-7b",
        use_fast=False,
    )

system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful?"

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')


tokenizer_config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [7]:
output_ids = model.generate(inputs["input_ids"],)
answer = tokenizer.batch_decode(output_ids)[0]

print(answer)

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful? <|im_end|> 
 <|im_start|> assistant
There are different ways to find out if a restaurant is popular among locals or mainly attracts tourists, and some possible reasons why this information might be useful are:

- If you are looking for a place to eat that offers authentic and affordable local cuisine, you might want to avoid the restaurants that cater more to tourists, who tend to prefer familiar and Westernized dishes.
- If you are a local business owner or a travel agent, you might want to know the preferences and habits of your target market, and use that information to promote your pr

In [8]:
# This example continues showing how to add a second turn message by the user to the conversation
second_turn_user_message = "Give me a list of the key points of your first answer."

# we set add_special_tokens=False because we dont want to automatically add a bos_token between messages
second_turn_message_in_markup = f"\n<|im_start|>user\n{second_turn_user_message}<|im_end|>\n<|im_start|>assistant"
second_turn_tokens = tokenizer(second_turn_message_in_markup, return_tensors='pt', add_special_tokens=False)
second_turn_input = torch.cat([output_ids, second_turn_tokens['input_ids']], dim=1)

output_ids_2 = model.generate(second_turn_input,)
second_turn_answer = tokenizer.batch_decode(output_ids_2)[0]

print(second_turn_answer)

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful? <|im_end|> 
 <|im_start|> assistant
There are different ways to find out if a restaurant is popular among locals or mainly attracts tourists, and some possible reasons why this information might be useful are:

- If you are looking for a place to eat that offers authentic and affordable local cuisine, you might want to avoid the restaurants that cater more to tourists, who tend to prefer familiar and Westernized dishes.
- If you are a local business owner or a travel agent, you might want to know the preferences and habits of your target market, and use that information to promote your pr